# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = ('../Output_data/Output_weather_data.csv')
vacation_file = pd.read_csv(data_file)
vacation_file.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,Clyde River,70.47,-68.59,44.60,87,40,8.05,CA,1596037471
1,1,Ürümqi,43.80,87.60,80.60,34,75,15.66,CN,1596037208
2,2,Bredasdorp,-34.53,20.04,66.20,55,0,13.87,ZA,1596037475
3,3,Puerto Madryn,-42.77,-65.04,47.77,52,0,11.83,AR,1596037477
4,4,Tasiilaq,65.61,-37.64,44.60,75,75,13.87,GL,1596037479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Storing lat/lng into locations
locations = vacation_file[["Lat", "Lng"]].astype(float)

#Convert humidity to float and store
humid_locale = vacation_file["Humidity"].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid_locale, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
under_80 = vacation_file[vacation_file['Max Temp']<=80]
under_80

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,Clyde River,70.47,-68.59,44.60,87,40,8.05,CA,1596037471
2,2,Bredasdorp,-34.53,20.04,66.20,55,0,13.87,ZA,1596037475
3,3,Puerto Madryn,-42.77,-65.04,47.77,52,0,11.83,AR,1596037477
4,4,Tasiilaq,65.61,-37.64,44.60,75,75,13.87,GL,1596037479
5,5,Souillac,-20.52,57.52,71.60,83,75,9.17,MU,1596037481
...,...,...,...,...,...,...,...,...,...,...
593,593,Ilhabela,-23.78,-45.36,68.11,86,100,7.34,BR,1596038609
594,594,Perth,-31.93,115.83,54.00,87,0,4.70,AU,1596038838
595,595,Teya,60.38,92.63,56.25,89,63,2.95,RU,1596038878
598,598,Barquisimeto,10.07,-69.32,68.00,100,15,10.29,VE,1596038673


In [7]:
not_windy = under_80[under_80['Wind Speed (m/s)']<1]
not_windy

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
130,130,Lebu,-37.62,-73.65,48.33,54,56,0.58,CL,1596037779
139,139,Jumla,29.27,82.18,62.47,72,100,0.96,NP,1596037801
173,173,Auki,-8.77,160.70,78.26,80,88,0.74,SB,1596037877
302,302,Azanka,58.04,64.79,56.12,69,97,0.98,RU,1596038194
350,350,Kuminskiy,58.67,66.57,56.19,65,75,0.78,RU,1596038305
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309
394,394,Baruun-Urt,46.68,113.28,61.05,84,17,0.87,MN,1596038409
467,467,Haines Junction,60.75,-137.51,51.98,83,100,0.92,CA,1596038583
509,509,Korem,12.51,39.52,61.12,85,95,0.94,ET,1596038407
592,592,Healesville,-37.65,145.52,48.99,99,47,0.36,AU,1596038872


In [8]:
hotel_df = not_windy[not_windy['Humidity']>=84]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309
394,394,Baruun-Urt,46.68,113.28,61.05,84,17,0.87,MN,1596038409
509,509,Korem,12.51,39.52,61.12,85,95,0.94,ET,1596038407
592,592,Healesville,-37.65,145.52,48.99,99,47,0.36,AU,1596038872


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# # Practice on one location

# target_coordinates = "-37.48, 144.57" 
# target_search = "lodging"
# target_radius = 5000
# target_type = "lodging"

# params = {
#     "location": target_coordinates,
#     "keyword": target_search,
#     "radius": target_radius,
#     "types": target_type,
#     "key": g_key
# }

# #API Call for nearest hotel
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# hotel_search = requests.get(base_url, params=params)
# # print(hotel_search.url)
# hotel_Healesville = hotel_search.json()
# # print(json.dumps(hotel_Healesville, indent=4, sort_keys=True))

In [ ]:
# hotel_Healesville["results"][0]["name"]

In [9]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309
394,394,Baruun-Urt,46.68,113.28,61.05,84,17,0.87,MN,1596038409
509,509,Korem,12.51,39.52,61.12,85,95,0.94,ET,1596038407
592,592,Healesville,-37.65,145.52,48.99,99,47,0.36,AU,1596038872


In [12]:
hotel_df['Hotel Name'] = " "
hotel_df

C:\Users\brook\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date,Hotel Name
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309,
394,394,Baruun-Urt,46.68,113.28,61.05,84,17,0.87,MN,1596038409,
509,509,Korem,12.51,39.52,61.12,85,95,0.94,ET,1596038407,
592,592,Healesville,-37.65,145.52,48.99,99,47,0.36,AU,1596038872,


In [ ]:
#Practice putting Hotel name in DF by df.loc method

# hotel_df.loc[352, 'Hotel Name'] = hotel_Tabuk["results"][0]["name"]
# hotel_df.loc[394, 'Hotel Name'] = hotel_Baruun["results"][0]["name"]
# hotel_df.loc[509, 'Hotel Name'] = hotel_Korem["results"][0]["name"]
# hotel_df.loc[592, 'Hotel Name'] = hotel_Healesville["results"][0]["name"]

In [15]:
# find the closest Hotel of each type to coordinates (use class activity "Nearest Restaurant" as help)
hotels = []
count = 0

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    target_coordinates = (f'{lat}, {lng}') 
    target_search = "lodging"
    target_radius = 5000
    target_type = "lodging"    
    
#Set up parameters
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make API request
    
    hotel_search = requests.get(base_url, params=params)
    hotel_data = hotel_search.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
        count=+1
        print(f'There is a hotel close to {count}.')
        
        
    except:
        count=+1
        print("You will not be vacationing at {count}.")
        
        print("------------")

There is a hotel close to 1.
There is a hotel close to 1.
You will not be vacationing at {count}.
------------
There is a hotel close to 1.


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness (%),Wind Speed (m/s),Country,Date,Hotel Name
352,352,Tabuk,17.42,121.44,75.25,87,100,0.34,PH,1596038309,Grand Zion Garden Resort Hotel
394,394,Baruun-Urt,46.68,113.28,61.05,84,17,0.87,MN,1596038409,"""Сод"" Буудал"
509,509,Korem,12.51,39.52,61.12,85,95,0.94,ET,1596038407,The Old Mechanics
592,592,Healesville,-37.65,145.52,48.99,99,47,0.36,AU,1596038872,Healesville House - Magnolia House


In [36]:
fig = gmaps.figure()

In [27]:
# Add markers
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))